In [1]:
import pandas
from pandas import DataFrame
import numpy
import matplotlib.pyplot as plt

In [2]:
%run Q_Data_Exploration.ipynb

So luong coin co trong data:  372
So luong coin con giao dich:  378


In [3]:
f_data = f_data
base_coin = base_coin
symbols = symbols

------

biến bảng trên thành giá trị quy đổi qua đồng tiền nền (Thực tế đã là bảng hiện tại) <br>
tạo 1 bảng giá tiền của đồng tiền theo symbols <br>
Cần đổi lại các đồng tiền có quote không nằm trong 'base_coin' theo quy tắc:
- base <-> quote
- Volume <-> Quote asset volume
- Taker buy base asset volume <-> Taker buy quote asset volume
- Các giá trị tiền tệ: New = 1 / Old

In [4]:
temp = f_data[~f_data.quote.isin(base_coin)]
f_data[~f_data.quote.isin(base_coin)] = pandas.concat([
    temp[['Symbol','quote','base' ,'Kline open time']],
    1/temp[['Open price','High price','Low price','Close price']],
    temp[['Quote asset volume','Kline Close time','Volume','Number of trades','Taker buy quote asset volume','Taker buy base asset volume','Unused field, ignore']]
],axis=1)
# f_data now is the data of "true value" of the token

C:\Users\TNT\AppData\Local\Temp\ipykernel_11180\949687093.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_data[~f_data.quote.isin(base_coin)] = pandas.concat([


Xem sự liên quan giữa giá và số lượng giao dịch

In [5]:
f_data[['base','Open price','High price','Low price','Close price','Number of trades']].groupby('base').mean().corr()

,Open price,High price,Low price,Close price,Number of trades
Open price,1.000000,1.000000,1.000000,1.000000,0.665169
High price,1.000000,1.000000,0.999999,1.000000,0.665176
Low price,1.000000,0.999999,1.000000,1.000000,0.665132
Close price,1.000000,1.000000,1.000000,1.000000,0.665142
Number of trades,0.665169,0.665176,0.665132,0.665142,1.000000


Độ tương quan khá thấp, nên có thể thấy số lượng giao dịch không có ảnh hưởng nhiều đến giá của đồng coin

---

Kiểm tra xem giá trị quy đổi giữa các token với nhau, có tuân theo giá trị thực không

In [6]:
data = pandas.read_csv('data.csv').drop(['Unnamed: 0'], axis=1) 

In [7]:
head_name = data.columns

In [72]:
def GetPrice(sym):
    for i in base_coin:
        temp = f_data[(f_data.base == sym) & (f_data.quote == i)]
        if temp.shape[0] !=0:
            return temp.set_index('Kline open time')[head_name[4:8]]
    return 1

In [130]:
temp = data.set_index(["Symbol","Kline open time"]).drop(head_name[8:],axis=1)
temp # xep lai data phan theo tung loại dao dich

base quote  Open price  High price  Low price  \
Symbol  Kline open time                                                  
ETHBTC  1669395600000     ETH   BTC    0.072026    0.072222   0.072026   
        1669397400000     ETH   BTC    0.072222    0.072335   0.072117   
        1669399200000     ETH   BTC    0.072163    0.072263   0.072101   
        1669401000000     ETH   BTC    0.072188    0.072261   0.072161   
        1669402800000     ETH   BTC    0.072223    0.072479   0.072197   
...                       ...   ...         ...         ...        ...   
BUSDRON 1670943600000    BUSD   RON    4.629000    4.630000   4.620000   
        1670945400000    BUSD   RON    4.620000    4.631000   4.620000   
        1670947200000    BUSD   RON    4.631000    4.631000   4.630000   
        1670949000000    BUSD   RON    4.630000    4.631000   4.630000   
        1670950800000    BUSD   RON    4.631000    4.631000   4.630000   

                         Close price  
Symbol  Kline open time               
ETHBTC  1669395600000       0.072222  
        1669397400000       0.072162  
        1669399200000       0.072189  
        1669401000000       0.072212  
        1669402800000       0.072375  
...                              ...  
BUSDRON 1670943600000       4.621000  
        1670945400000       4.630000  
        1670947200000       4.631000  
        1670949000000       4.631000  
        1670950800000       4.631000  

[1229162 rows x 6 columns]

In [127]:
diff = []
for i in set(temp.index.get_level_values(0)):
    baseA  =  symbols[symbols.symbol==i].baseAsset.values[0]
    quoteA = symbols[symbols.symbol==i].quoteAsset.values[0]
    cal = GetPrice(baseA) / GetPrice(quoteA)
    try:
        gap = numpy.nanmean(cal - temp.loc[i][head_name[4:8]],axis=0) #differenct between price
        #normalnize
        gap /= numpy.mean(temp.loc[i][head_name[4:8]],axis=0)
        diff += [gap]        
    except:
        diff += [[0,0,0,0]]
    pass

In [128]:
numpy.mean(diff,axis = 0)

array([0.02731673, 0.02641888, 0.02802794, 0.02729723])

Sự chênh lệch nằm ở tầm 2% (dương) giá trị token, có thể nói khi trao đổi thông qua đồng tiền nền thì ta sẽ mất nhiều phí hơn là trao đổi trực tiếp giữa 2 token với nhau, nhưng đồng thời cũng cho thấy được là giá trị quy đổi các token có tuân theo giá trị thực của chúng

------